# Train and Run the Underlying MLP Models in TELL Using Evolving Time Windows

This notebook is the core tool for training and running the evolving sets of multilayer perceptron (MLP) models used in this experiment. It assumes that the base Total ELectricity Loads (TELL) model has already been installed and the datasets updated to include the most recent data.

In [ ]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell

import pandas as pd


## Set the Directory Structure


In [ ]:
# Identify the top-level directory to store the trained MLP models:
ba_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/tell/tell/tell_data/tell_quickstarter_data/outputs/compiled_historical_data/'
weather_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/tell/tell/tell_data/sample_forcing_data/historical_weather'
model_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/trained_mlp_models/'
prediction_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/mlp_projections/'
composite_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/composite_projections/'


## MLP Model Training

The MLP models underpinning TELL use temporal variations in weather to project hourly demand. More information about this approach is in the MLP section of the `tell` [User Guide](https://immm-sfa.github.io/tell/user_guide.html). The default settings for the MLP model training steps are included in the `mlp_settings.yml` file included in the data folder of the `tell` repository. By default the MLP models are trained on data from 2016-2018 and evaluated using data from 2019. The time windows for training and evaluating the models can be modified by altering the `start_time`, `end_time`, and `split_datetime` parameters when calling the `tell.train` function. The workflow does not change the default MLP parameters (e.g., hidden layer sizes, maximum number of iterations, etc.) from TELL. Those could be modified in theory.

In [ ]:
# For more information about the training of predictive models you can call the help function:
help(tell.train)


In [ ]:
# Run the MLP training step for a single BA to get a feel of the functionality:
prediction_df, validation_df = tell.train(region = 'CISO',
                                          data_dir = '/Users/burl878/Documents/Code/code_repos/tell/tell/tell_data/tell_quickstarter_data/outputs/compiled_historical_data/',
                                          start_time = '2018-01-01 00:00:00',
                                          end_time = '2020-12-31 23:00:00',
                                          split_datetime = '2019-12-31 23:00:00',
                                          save_model = True,
                                          model_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/trained_mlp_models/M1/')

# View the head of the prediction dataframe that contains the time-series of projected load in the evaluation year:
display(prediction_df.head(10))

# View validation dataframe that contains error statistics for the trained model:
validation_df


In [ ]:
# Run the training iteratively across all training windows and BAs:
for model in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']:

    # Set the training and evaluation periods for the specific model training window:
    if model == 'M1':
       start_time_model = '2016-01-01 00:00:00'
       end_time_model = '2018-12-31 23:00:00'
       split_datetime_model = '2017-12-31 23:00:00'
    if model == 'M2':
       start_time_model = '2017-01-01 00:00:00'
       end_time_model = '2019-12-31 23:00:00'
       split_datetime_model = '2018-12-31 23:00:00'
    if model == 'M3':
       start_time_model = '2018-01-01 00:00:00'
       end_time_model = '2020-12-31 23:00:00'
       split_datetime_model = '2019-12-31 23:00:00'
    if model == 'M4':
       start_time_model = '2019-01-01 00:00:00'
       end_time_model = '2021-12-31 23:00:00'
       split_datetime_model = '2020-12-31 23:00:00'
    if model == 'M5':
       start_time_model = '2020-01-01 00:00:00'
       end_time_model = '2022-12-31 23:00:00'
       split_datetime_model = '2021-12-31 23:00:00'
    if model == 'M6':
       start_time_model = '2021-01-01 00:00:00'
       end_time_model = '2023-12-31 23:00:00'
       split_datetime_model = '2022-12-31 23:00:00'

    # Create the model output directory name:
    output_directory = (model_output_directory + model + '/')
  
    # Check to see if the output directory exist and if not then create it:
    if not os.path.exists(output_directory):
       os.makedirs(output_directory)

    # Loop over the eight BAs used in this LDRD analysis:
    for ba in ['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'WACM']:
        
        # Run the MLP training for that BA and model training period:
        prediction_df, validation_df = tell.train(region = ba,
                                                  data_dir = ba_data_input_dir,
                                                  start_time = start_time_model,
                                                  end_time = end_time_model,
                                                  split_datetime = split_datetime_model,
                                                  save_model = True,
                                                  model_output_directory = output_directory)

        # Print the model and BA combination to monitor the progress:
        print('Model = ', model, ', BA = ', ba)


## Project Forward Using the Trained MLP Models

Use the TELL forward projection function to run the trained MLP models forward in time to project "future" loads.

In [ ]:
# For more information about the how to use the models to project loads forward in time you can call the help function:
help(tell.predict)


In [ ]:
# Run the MLP forward projection step for a single BA to get a feel of the functionality:
pdf = tell.predict(region = 'CISO',
                   year = 2023,
                   data_dir = weather_data_input_dir,
                   datetime_field_name = 'Time_UTC',
                   save_prediction = True,
                   model_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/trained_mlp_models/M1',
                   prediction_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/mlp_projections/M1')

pdf


In [ ]:
# Run the forward projection step iteratively across all training windows and BAs:
for model in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']:

    # Set the first forward year for each model:
    if model == 'M1':
       first_forward_year = 2018
    if model == 'M2':
       first_forward_year = 2019
    if model == 'M3':
       first_forward_year = 2020
    if model == 'M4':
       first_forward_year = 2021
    if model == 'M5':
       first_forward_year = 2022
    if model == 'M6':
       first_forward_year = 2023

    # Create the model output directory name:
    output_directory = (model_output_directory + model + '/')
  
    # Check to see if the output directory exist and if not then create it:
    if not os.path.exists(output_directory):
       os.makedirs(output_directory)

    # Loop over the eight BAs used in this LDRD analysis:
    for ba in ['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'WACM']:

        # Loop over the years from the first forward year for that model through 2023:
        for year in range(first_forward_year,2024,1):
        
            # Run the MLP forward projection for that BA, model, and year:
            pdf = tell.predict(region = ba,
                               year = year,
                               data_dir = weather_data_input_dir,
                               datetime_field_name = 'Time_UTC',
                               save_prediction = True,
                               model_output_directory = (model_output_directory + model + '/'),
                               prediction_output_directory = (prediction_output_directory + model + '/'))

            # Print the model and BA combination to monitor the progress:
            print('Model = ', model, ', BA = ', ba, ', Year = ', str(year))


## Compile the Historical and Forward Projection Datasets into a Single .csv File for Each BA


In [ ]:
# Loop over the eight BAs used in this LDRD analysis:
for ba in ['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'WACM']:

    # Load in the compiled historical data from the TELL repository:
    base_df = pd.read_csv((ba_data_input_dir + ba + '_historical_data.csv'), index_col=None, header=0)

    # Convert the time columns into one datetime variable:
    base_df['Time_UTC'] = pd.to_datetime(base_df[['Year', 'Month', 'Day', 'Hour']])
    
    # Convert the temperature from Kelvin to Fahrenheit:
    base_df['T2'] = ((1.8 * (base_df['T2'] - 273)) + 32).round(2)

    # Round the population to the nearest whole integer and demand to the nearest tenth:
    base_df['Adjusted_Demand_MWh'] = base_df['Adjusted_Demand_MWh'].round(1)
    base_df['Total_Population'] = base_df['Total_Population'].round(0).astype(int)
    
    # Rename the demand and population columns to something simpler:
    base_df.rename(columns={'Adjusted_Demand_MWh': 'Demand_MWh', 'Total_Population': 'Population'}, inplace=True)

    # Add in the BA code for better tracking:
    base_df['BA'] = ba
    
    # Only keep the columns that are needed:
    base_df = base_df[['BA', 'Time_UTC', 'T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD', 'Population', 'Demand_MWh']].copy()

    # Loop over the different models and merge in their demand projections iteratively:
    for model in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']:

        # Set the first forward year for each model:
        if model == 'M1':
           first_forward_year = 2018
        if model == 'M2':
           first_forward_year = 2019
        if model == 'M3':
           first_forward_year = 2020
        if model == 'M4':
           first_forward_year = 2021
        if model == 'M5':
           first_forward_year = 2022
        if model == 'M6':
           first_forward_year = 2023

        # Loop over the years from the first forward year for that model through 2023:
        for year in range(first_forward_year,2024,1):

            # Load in the MLP projection for that model and year combination:
            proj_df = pd.read_csv((prediction_output_directory + model + '/' + str(year) + '/' + ba  + '_' + str(year) + '_mlp_output.csv'), index_col=None, header=0)

            # Only keep the columns that are needed:
            proj_df = proj_df[['Time_UTC', 'Load']].copy()

            # Round the demand to the nearest tenth:
            proj_df['Load'] = proj_df['Load'].round(1)
    
            # Rename the load variable to reflect the model used to generate the projection:
            proj_df.rename(columns={'Load': (model + '_MWh')}, inplace=True)

            # Convert the time to a datetime variable:
            proj_df['Time_UTC'] = pd.to_datetime(proj_df['Time_UTC'])

            # Aggregate the output into a new dataframe:
            if year == first_forward_year:
               aggregate_proj_df = proj_df
            else:
               aggregate_proj_df = pd.concat([aggregate_proj_df, proj_df])
            
        # Merge the base_df and proj_df dataframes using common UTC times:
        base_df = base_df.merge(aggregate_proj_df, on=['Time_UTC'], how='left')

    # Copy the data into an output dataframe:
    output_df = base_df.copy()

    # Replace NaN values with -999:
    output_df = output_df.fillna(-999)

    # Set the output file name:
    csv_output_filename = (composite_output_directory + ba + '_Composite_Data.csv')

    # Write out the dataframe to a .csv file:
    output_df.to_csv(csv_output_filename, sep=',', index=False)

    # Clean up the variables and move to the next BA in the loop:
    del base_df, model, first_forward_year, year, proj_df, aggregate_proj_df, output_df, csv_output_filename
